<a href="https://colab.research.google.com/github/RCCyber/RCCyber/blob/main/detect_employees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Activation, Flatten, Dropout

from keras import backend as K
K.image_dim_ordering='th'

import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt

In [ ]:
!cd /content/sample_data/ && unzip data.zip

In [4]:
inc_model=InceptionV3(include_top=False, 
                      weights='imagenet', 
                      input_shape=((116, 116, 3)))

87924736/87910968 [==============================] - 1s 0us/step


In [5]:
bottleneck_datagen = ImageDataGenerator(rescale=1./255)  #собственно, генератор
    
train_generator = bottleneck_datagen.flow_from_directory('/content/sample_data/data/train/',
                                        target_size=(116, 116),
                                        batch_size=32,
                                        class_mode=None,
                                        shuffle=False)

validation_generator = bottleneck_datagen.flow_from_directory('/content/sample_data/data/validation/', 
                                                              target_size=(116, 116),
                                                               batch_size=32,
                                                               class_mode=None,
                                                               shuffle=False)

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
bottleneck_features_train = inc_model.predict(train_generator, 2000)
np.save(open('bn_features_train.npy', 'wb'), bottleneck_features_train)

bottleneck_features_validation = inc_model.predict(validation_generator, 2000)
np.save(open('bn_features_validation.npy', 'wb'), bottleneck_features_validation)

In [7]:
train_data = np.load(open('bn_features_train.npy', 'rb'))
train_labels = np.array([0] * 1000 + [1] * 1000) 

validation_data = np.load(open('bn_features_validation.npy', 'rb'))
validation_labels = np.array([0] * 1000 + [1] * 1000)

In [8]:
fc_model = Sequential()
fc_model.add(Flatten(input_shape=train_data.shape[1:]))
fc_model.add(Dense(64, activation='relu', name='dense_one'))
fc_model.add(Dropout(0.5, name='dropout_one'))
fc_model.add(Dense(64, activation='relu', name='dense_two'))
fc_model.add(Dropout(0.5, name='dropout_two'))
fc_model.add(Dense(1, activation='sigmoid', name='output'))

fc_model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [9]:
#validation_labels = validation_labels.reshape(1, -1)
#train_labels = train_labels.reshape(1, -1)
print(validation_data.shape)
print(validation_labels.shape)
print(train_data.shape)
print(train_labels.shape)

(2000, 2, 2, 2048)
(2000,)
(2000, 2, 2, 2048)
(2000,)


In [ ]:
fc_model.fit(train_data, train_labels, epochs=50, batch_size=32, validation_data=(validation_data, validation_labels))
fc_model.save_weights('fc_inception_cats_dogs_250.hdf5') # сохраняем веса

In [13]:
fc_model.evaluate(validation_data, validation_labels)

63/63 [==============================] - 0s 4ms/step - loss: 1.3855 - accuracy: 0.9315


[1.3854981660842896, 0.9315000176429749]

In [14]:
weights_filename='fc_inception_cats_dogs_250.hdf5'
x = Flatten()(inc_model.output)
x = Dense(64, activation='relu', name='dense_one')(x)
x = Dropout(0.5, name='dropout_one')(x)
x = Dense(64, activation='relu', name='dense_two')(x)
x = Dropout(0.5, name='dropout_two')(x)
top_model=Dense(1, activation='sigmoid', name='output')(x)
model = Model(inputs=inc_model.input, outputs=top_model)

In [16]:
weights_filename='fc_inception_cats_dogs_250.hdf5'
model.load_weights(weights_filename, by_name=True)

In [17]:
for layer in inc_model.layers[:205]:
    layer.trainable = False

In [19]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=1e-4, momentum=0.9),
                #optimizer='rmsprop',
              metrics=['accuracy'])

In [32]:
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [35]:
train_datagen = ImageDataGenerator(
        rescale=1./255)#,
        #shear_range=0.2,
        #zoom_range=0.2,
        #horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/sample_data/data/train/',
        target_size=(116, 116),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        '/content/sample_data/data/validation/',
        target_size=(116, 116),
        batch_size=32,
        class_mode='binary')


pred_generator=test_datagen.flow_from_directory('/content/sample_data/data/validation/',
                                                     target_size=(116,116),
                                                     batch_size=100,
                                                     class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [36]:
model.fit(
        train_generator,
        steps_per_epoch=2000,
        epochs=200,
        validation_data=validation_generator,
        #nb_val_samples=2000,
    callbacks=callbacks_list)

Epoch 1/200
  62/2000 [..............................] - ETA: 1:30 - loss: 0.0403 - accuracy: 0.9909WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 400000 batches). You may need to use the repeat() function when building your dataset.


KeyError: ignored

In [ ]:
model.evaluate_generator(pred_generator, val_samples=100)

In [ ]:
mgs,labels=pred_generator.next()
array_imgs=np.transpose(np.asarray([img_to_array(img) for img in imgs]),(0,2,1,3))
predictions=model.predict(imgs)
rounded_pred=np.asarray([round(i) for i in predictions])

In [ ]:
wrong=[im for im in zip(array_imgs, rounded_pred, labels, predictions) if im[1]!=im[2]]

plt.figure(figsize=(12,12))
for ind, val in enumerate(wrong[:100]):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1, wspace = 0.2, hspace = 0.2)
    plt.subplot(5,5,ind+1)
    im=val[0]
    plt.axis('off')
    plt.text(120, 0, round(val[3], 2), fontsize=11, color='red')
    plt.text(0, 0, val[2], fontsize=11, color='blue')
    plt.imshow(np.transpose(im,(2,1,0)))

In [ ]:
right=[im for im in zip(array_imgs, rounded_pred, labels, predictions) if im[1]==im[2]]

plt.figure(figsize=(12,12))
for ind, val in enumerate(right[:20]):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1, wspace = 0.2, hspace = 0.2)
    plt.subplot(5,5,ind+1)
    im=val[0]
    plt.axis('off')
    plt.text(120, 0, round(val[3], 2), fontsize=11, color='red')
    plt.text(0, 0, val[2], fontsize=11, color='blue')
    plt.imshow(np.transpose(im,(2,1,0)))